In [153]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import display, HTML, Image, SVG
import os
import math
import timeit
from io import StringIO
import re
import urllib.request, json

print("pandas version: {} numpy version: {}".format(pd.__version__, np.__version__))

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

print("matplotlib version: {} seaborn version: {}".format(matplotlib.__version__, sns.__version__))

import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
# Check core SDK version number
print("azureml SDK version:", azureml.core.VERSION)

pandas version: 0.25.3 numpy version: 1.18.5
matplotlib version: 3.2.1 seaborn version: 0.11.1
azureml SDK version: 1.22.0


In [155]:
url = ("https://quickstartsws9073123377.blob.core.windows.net/"
       "azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/"
       "COVID19_Case_Surveillance_Data/COVID-19_Case_Surveillance_Public_Use_Data.csv")

In [156]:
df = pd.read_csv(url)
len(df)

8405079

In [157]:
df.columns

Index(['cdc_report_dt', 'pos_spec_dt', 'onset_dt', 'current_status', 'sex',
       'age_group', 'Race and ethnicity (combined)', 'hosp_yn', 'icu_yn',
       'death_yn', 'medcond_yn'],
      dtype='object')

In [170]:
df['datekey'] =  pd.to_datetime(df['cdc_report_dt'])

df['datekey'] = df['datekey'].dt.strftime('%Y-%m-%d')

In [186]:
df['death'] = df['death_yn']

In [187]:
dfgroup = df.groupby(['datekey','age_group','death_yn']).size().to_frame()

dfflat = dfgroup.reset_index()

In [188]:

dfgroup = df.groupby(['datekey']).size().to_frame()
dfflat.head(10)

,datekey,age_group,death_yn,0
0,2020-01-01,20 - 29 Years,No,1
1,2020-01-01,30 - 39 Years,No,3
2,2020-01-01,40 - 49 Years,No,1
3,2020-01-01,40 - 49 Years,Unknown,1
4,2020-01-01,50 - 59 Years,No,2
5,2020-01-01,60 - 69 Years,No,2
6,2020-01-01,70 - 79 Years,No,2
7,2020-01-02,0 - 9 Years,Missing,1
8,2020-01-02,30 - 39 Years,No,1
9,2020-01-02,50 - 59 Years,No,1


In [189]:
from pandas.io.json import json_normalize

dfflat['count'] = dfflat[0].apply(str)
dfflat = dfflat.drop(columns=[0])

In [191]:
for row in dfgroup.axes[0]:
    print(row)
    is_date =  dfflat['datekey']==row
    df_date = dfflat[is_date]
    resultJSON = df_date.to_json(orient='records', date_format='%Y-%m-%d')
    filename_processed_json =  f'covid_age_death/{row}.json'
    with open(filename_processed_json, 'w') as f:
        f.write(resultJSON)

2020-01-01
(7, 4)
covid_age_death/2020-01-01.json
2020-01-02
(3, 4)
covid_age_death/2020-01-02.json
2020-01-03
(1, 4)
covid_age_death/2020-01-03.json
2020-01-05
(1, 4)
covid_age_death/2020-01-05.json
2020-01-08
(1, 4)
covid_age_death/2020-01-08.json
2020-01-09
(2, 4)
covid_age_death/2020-01-09.json
2020-01-10
(2, 4)
covid_age_death/2020-01-10.json
2020-01-11
(5, 4)
covid_age_death/2020-01-11.json
2020-01-12
(1, 4)
covid_age_death/2020-01-12.json
2020-01-13
(2, 4)
covid_age_death/2020-01-13.json
2020-01-14
(13, 4)
covid_age_death/2020-01-14.json
2020-01-15
(5, 4)
covid_age_death/2020-01-15.json
2020-01-16
(3, 4)
covid_age_death/2020-01-16.json
2020-01-17
(6, 4)
covid_age_death/2020-01-17.json
2020-01-18
(4, 4)
covid_age_death/2020-01-18.json
2020-01-19
(3, 4)
covid_age_death/2020-01-19.json
2020-01-20
(6, 4)
covid_age_death/2020-01-20.json
2020-01-21
(13, 4)
covid_age_death/2020-01-21.json
2020-01-22
(4, 4)
covid_age_death/2020-01-22.json
2020-01-23
(7, 4)
covid_age_death/2020-01-23.jso

In [192]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "fdbba0bc-f686-4b8b-8b29-394e0d9ae697"

#Provide values for the existing Resource Group 
resource_group = "mcw-support-jss"

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "mcwmachinelearning"
workspace_region = "eastus"

In [193]:
ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name=workspace_name)



In [209]:
blob_account_key = "Fvj1aPaFMb86L6A+pzKgciDQAzjhlLOCA3EIcKzoSGOHR1pO6AB0cTCv7JQmF+qkHidkK5C8XxfyMvwvXJ0DZA=="
blob_datastore_name='covid_datastore_age_death' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "jsonmetrics-age-death") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "mcwmachinelear1659324051") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", blob_account_key) # Storage account access key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [195]:
#upload json files to the blob storage container
blob_datastore.upload('./covid_age_death', 
                 target_path = '', 
                 overwrite = True, 
                 show_progress = True)

Uploading an estimated of 321 files
Uploading ./covid_age_death/2020-01-01.json
Uploaded ./covid_age_death/2020-01-01.json, 1 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-02.json
Uploaded ./covid_age_death/2020-01-02.json, 2 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-03.json
Uploaded ./covid_age_death/2020-01-03.json, 3 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-05.json
Uploaded ./covid_age_death/2020-01-05.json, 4 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-08.json
Uploaded ./covid_age_death/2020-01-08.json, 5 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-09.json
Uploaded ./covid_age_death/2020-01-09.json, 6 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-10.json
Uploaded ./covid_age_death/2020-01-10.json, 7 files out of an estimated total of 321
Uploading ./covid_age_death/2020-01-11.json
Uploaded ./covid_

$AZUREML_DATAREFERENCE_covid_datastore_age_death

In [196]:
pip install azure-ai-metricsadvisor --pre

     |████████████████████████████████| 127 kB 15.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Optional steps: Create Azure Blob Data Feed (manually in this lab)

In [225]:
subscription_key = "2efb3914e02e41e389f951a5e4bd618f" #from Azure portal, metrics advisor created resource , from the left menu go to Keys and Endpoint page
api_key = "fcc627d8-5bcb-4742-9979-98ecad173971" #from the metrics advisor portal, from the left menu go to API keys page
service_endpoint = "https://mcw-metricsadvisor.cognitiveservices.azure.com/"
client = MetricsAdvisorAdministrationClient(service_endpoint,MetricsAdvisorKeyCredential(subscription_key, api_key))
client

<MetricsAdvisorAdministrationClient [endpoint='https://mcw-metricsadvisor.cognitiveservices.azure.com/']>

In [233]:
from azure.ai.metricsadvisor import MetricsAdvisorKeyCredential, MetricsAdvisorAdministrationClient
from azure.ai.metricsadvisor.models import (
        AzureBlobDataFeed,
        DataFeedSchema,
        DataFeedMetric,
        DataFeedDimension,
        DataFeedOptions,
        DataFeedRollupSettings,
        DataFeedMissingDataPointFillSettings
    )
storage_accountkey = "1NQ8HlB3V9v+XAQv1OsxfbJt5DHMDltgCVlcEInKthKNyT1bg97JU4VXDeli/INgE18RWdNrjReWyP5tVVBTpA=="
blobstorage_connection_string = f"DefaultEndpointsProtocol=https;AccountName=mcwmachinelear1659324051;AccountKey={storage_accountkey};EndpointSuffix=core.windows.net"
blob_container = "jsonmetrics-age-death"
blobtemplate = "%Y-%m-%d.json"
#templateversion = "v2"

client = MetricsAdvisorAdministrationClient(service_endpoint,MetricsAdvisorKeyCredential(subscription_key, api_key))

data_feed = client.create_data_feed(
    name="covid-dailybyages-feed",
    source=AzureBlobDataFeed(
        connection_string = blobstorage_connection_string, 
        container = blob_container, 
        blob_template = blobtemplate,
        #missing jsonFormatVersion = v2 (property is not exposed in the constructor)
    ),
    
    granularity="Daily",
    schema=DataFeedSchema(
        metrics=[
            DataFeedMetric(name="count", display_name="Count")
        ],
        dimensions=[
            DataFeedDimension(name="age_group", display_name="Age Group"),
            DataFeedDimension(name="death_yn", display_name="Death")
        ]
    ),
    ingestion_settings=datetime.datetime(2020, 1, 1),
    options=DataFeedOptions(
        data_feed_description="cases by age data feed",
        rollup_settings=DataFeedRollupSettings(
            rollup_type="AutoRollup",
            rollup_method="Sum",
            rollup_identification_value="__CUSTOM_SUM__"
        ),
        missing_data_point_fill_settings=DataFeedMissingDataPointFillSettings(
            fill_type="SmartFilling"
        )
    )
)

Datetime with no tzinfo will be considered UTC.
